<a href="https://colab.research.google.com/github/FernandoElizarraras7/SIMULACION-II/blob/main/MUESTREO_IMPORTANCIA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# MUESTREO DE IMPORTANCIA
Es una técnica de reducción de varianza que mejora la precisión de las estimaciones de Monte Carlo. Su objetivo principal es mejorar la precisión de la estimación al concentrar más muestras en las zonas donde la función que queremos integrar tiene mayor peso.

En lugar de generar muestras de una distribución uniforme, se utiliza una distribución auxiliar $g(x)$ que se asemeje a la forma de la función $f(x)$ que se desea integrar.

Suponga que queremos estimar una integral del tipo:
$$I= \int_{a}^{b}f(x)dx$$
Si usamos Monte Carlo crudo, generamos $x_i \sim U(a,b)$ (uniformes) y estimamos:
$$I ≈ (b-a) \frac{1}{N} \sum_{i=1}^{N} f(x_i)$$
Sin embargo, esto puede tener una varianza alta si $f(x)$ varía mucho.
El muestreo de importancia propone usar una distribución diferente $g(x)$ que se parezca más a la forma de $f(x)$, y reescribir la integral como:
$$I= \int_{a}^{b} \frac{f(x)}{g(x)} g(x)dx = E_g \left [ \frac{f(x)}{g(x)}  \right ]$$
donde $x_i \sim g(x)$.
Así, la estimación de Monte Carlo se convierte en:
$$I_{IS} = \frac{1}{N} \sum_{i=1}^{N} \frac{f(x_i)}{g(x_i)}$$

### VENTAJAS
*   Si $g(x)$ se elige de modo que sea proporcional a $f(x)$, la varianza del estimador disminuye notablemente.
*   Esto se debe a que las muestras se concentran en las regiones donde $f(x)$ contribuye más a la integral.
*   En la práctica, $g(x)$ debe cumplir:
    1.   $g(x) > 0$ cuando $f(x) \neq 0$
    2.   $\int g(x) dx = 1$












In [1]:
import numpy as np
import time

# Función objetivo f(x)
def f(x):
  return np.cos(np.pi * x / 2)

# Función de densidad g(x)
def g(x):
  return (24 / (24 - np.pi**2)) * (1-(np.pi**2 * x**2) / 8)

# Número de muestras
N = 10**5

# === Monte Carlo Crudo ===
t1_start = time.time()

x_crudo = np.random.rand(N)  # Muestras en [0,1]
fx = f(x_crudo)  # Se evalúa en f(x)
I_crudo = np.mean(fx)  # Estimador de la integral
Var_crudo = np.var(fx, ddof=1) / N  # Varianza del estimador

t1_end = time.time()
t_crudo = t1_end - t1_start

print("=== Método Crudo ===")
print(f"Estimación Integral = {I_crudo:.6f}")
print(f"Varianza = {Var_crudo:.6e}")
print()

# === Muestreo de importancia ===
t2_start = time.time()

# Se van a generar muestras de x ~ g(x)
# Se usa el método de aceptación-rechazo para generar muestras según g(x)
# Ya que g(x) no tiene una inversa simple

# Máximo de g(x) en [0,1] (para aceptación-rechazo)
x_grid = np.linspace(0, 1, 10000)
g_max = np.max(g(x_grid))

samples = []
while len(samples) < N:
    x_try = np.random.rand(N) # Candidatos uniformes
    y_try = np.random.rand(N) * g_max # Valores en [0, g_max]
    mask = y_try <= g(x_try) # Aceptación-rechazo
    accepted = x_try[mask]
    samples.extend(accepted.tolist())

x_imp = np.array(samples[:N]) # Se toman N muestras

# Se evaluan f y g
fx = f(x_imp)
gx = g(x_imp)

# Estimador del muestreo de importancia
ponderaciones = fx / gx
I_imp = np.mean(ponderaciones)
Var_imp = np.var(ponderaciones, ddof=1) / N

t2_end = time.time()
t_imp = t2_end - t2_start

print("=== Método de Muestreo de Importancia ===")
print(f"Estimación Integral = {I_imp:.6f}")
print(f"Varianza = {Var_imp:.6e}")
print()

# === Comparacion ===
reduction_pct = 100 * (Var_crudo - Var_imp) / Var_crudo
print("=== Comparación ===")
print(f"Reducción relativa de varianza = {reduction_pct:.2f}%")

=== Método Crudo ===
Estimación Integral = 0.636575
Varianza = 9.443408e-07

=== Método de Muestreo de Importancia ===
Estimación Integral = 0.615987
Varianza = 2.057428e-07

=== Comparación ===
Reducción relativa de varianza = 78.21%


## CONCLUSIONES
*   El método de muestreo de importancia permitió una reducción significativa de la varianza del estimador (78.21%), lo que se traduce en estimaciones más precisas sin necesidad de incrementar el número de simulaciones.
*   La elección adecuada de la distribución de importancia es clave; al enfocar la simulación en las regiones que más contribuyen al valor esperado, se mejora la eficiencia del método.
*   Comparado con el Monte Carlo estándar, el muestreo de importancia demuestra ser una técnica muy eficaz para problemas donde algunas regiones del dominio dominan el resultado, optimizando tiempo de cómputo y recursos.
*   Estos resultados confirman que el muestreo de importancia es una herramienta poderosa para la integración numérica y estimaciones bajo incertidumbre, especialmente en funciones con alta variabilidad en ciertas regiones del dominio.



